In energy production, the power grid is a a large directed graph of energy consumers and producers. At times you need to cut at certain nodes and trim demand because you cannot supply enough of a load.

In DailyProgrammeropolis, all buildings are connected to the grid and all consume power to varying degrees. Some generate power because they have installed on-site generation and sell the excess to the grid, some do not.

The scenario you're facing is this: due to a fault with the bulk power generation facility not local to DailyProgrammerololis, you must trim the power grid. You have connectivity data, and power consumption and production data. Your goal with this challenge is to maximize the number of powered nodes with the generated energy you have. Note that when you cut off a node, you run the risk the downstream ones will loose power, too, if they are no longer connected. This is how you'll shed demand, by selectively cutting the graph. You can make as many cuts as you want (there is no restriction on this). 

In [1]:
nodes = """0 1.926 0.0
1 36.812 0.0
2 1.007 0.0
3 6.812 0.0
4 1.589 0.0
5 1.002 0.0
6 1.531 0.0
7 2.810 0.0
8 1.246 0.0
9 5.816 0.0
10 1.167 0.0
11 1.357 0.0
12 1.585 0.0
13 1.117 0.0
14 3.110 1.553
15 2.743 0.0
16 1.282 0.0
17 1.154 0.0
18 1.160 0.0
19 1.253 0.0
20 1.086 0.0
21 1.148 0.0
22 1.357 0.0
23 2.161 0.0
24 1.260 0.0
25 2.241 0.0
26 2.970 0.0
27 6.972 0.0
28 2.443 0.0
29 1.255 0.0
30 1.844 0.0
31 2.503 0.0
32 1.054 0.0
33 1.368 0.0
34 1.011 1.601
35 1.432 0.0
36 1.061 1.452
37 1.432 0.0
38 2.011 0.0
39 1.232 0.0
40 1.767 0.0
41 1.590 0.0
42 2.453 0.0
43 1.972 0.0
44 1.445 0.0
45 1.197 0.0
46 2.497 0.0
47 3.510 0.0
48 12.510 0.0
49 3.237 0.0
50 1.287 0.0
51 1.613 0.0
52 1.776 0.0
53 2.013 0.0
54 1.079 0.0
55 1.345 1.230
56 1.613 0.0
57 2.243 0.0
58 1.209 0.0
59 1.429 0.0
60 7.709 0.0
61 1.282 8.371
62 1.036 0.0
63 1.086 0.0
64 1.087 0.0
65 1.000 0.0
66 1.140 0.0
67 1.210 0.0
68 1.080 0.0
69 1.087 0.0
70 1.399 0.0
71 2.681 0.0
72 1.693 0.0
73 1.266 0.0
74 1.234 0.0
75 2.755 0.0
76 2.173 0.0
77 1.093 0.0
78 1.005 0.0
79 1.420 0.0
80 1.135 0.0
81 1.101 0.0
82 1.187 1.668
83 2.334 0.0
84 2.054 3.447
85 1.711 0.0
86 2.083 0.0
87 2.724 0.0
88 1.654 0.0
89 1.608 0.0
90 1.033 17.707
91 1.017 0.0
92 1.528 0.0
93 1.278 0.0
94 1.128 0.0
95 1.508 1.149
96 5.123 0.0
97 2.000 0.0
98 1.426 0.0
99 1.802 0.0
100 2.995 98.606""".split('\n')

In [2]:
import pandas as pd
nodes = [i.split() for i in nodes]

power_usage = {}
power_generation = {}

for node in nodes:
    power_usage[node[0]] = node[1]
    power_generation[node[0]] = node[2]

In [3]:
edges = '''5 0
5 3
6 1
6 3
7 6
8 4
9 3
10 9
10 3
11 6
12 5
12 7
13 2
14 10
14 11
14 13
15 9
15 13
15 14
16 5
16 14
17 1
18 8
18 16
18 13
19 0
19 3
19 4
19 6
20 16
20 5
20 14
21 0
21 17
21 19
21 12
21 14
22 3
22 20
23 0
23 1
23 11
23 21
23 7
24 12
24 22
24 6
24 15
25 0
25 4
25 14
26 3
27 2
27 10
28 0
29 17
30 3
30 4
30 6
30 15
30 17
30 26
30 29
31 17
31 10
31 22
32 29
33 16
33 10
33 4
33 29
34 0
34 1
34 28
34 30
35 8
35 24
35 5
35 30
36 8
36 12
36 7
37 16
37 24
37 6
38 22
38 7
39 1
39 3
39 8
39 10
39 17
39 29
40 9
40 26
40 12
41 0
42 0
42 33
42 14
42 16
42 19
42 25
43 32
43 2
43 37
43 38
43 39
43 16
43 17
43 29
44 35
44 4
44 9
44 43
44 18
44 27
45 7
45 40
45 41
45 11
45 24
45 26
46 32
46 42
46 29
47 32
47 34
47 36
47 38
47 8
47 10
47 14
47 18
47 24
48 33
48 6
48 39
48 8
48 7
48 44
48 15
48 17
48 47
49 1
49 33
49 8
49 47
49 40
49 22
49 29
50 40
50 19
50 3
50 37
50 27
51 4
51 38
51 43
51 13
51 23
51 26
52 1
52 5
52 45
52 49
52 50
52 24
52 28
53 5
53 40
53 9
53 15
53 27
53 30
54 43
54 4
54 7
54 8
54 10
54 11
54 51
54 20
54 27
55 32
55 48
55 18
55 4
55 14
56 45
56 47
56 48
56 49
56 55
56 24
57 7
57 42
57 11
57 13
57 46
57 48
57 21
57 55
58 32
58 39
58 10
58 48
58 52
58 22
58 23
58 26
59 1
59 36
59 37
59 8
59 18
59 22
59 31
60 32
60 35
60 8
60 15
60 40
60 50
60 21
61 1
61 34
61 35
61 4
61 40
61 46
61 57
61 58
62 14
62 7
63 9
63 55
63 46
63 39
64 32
64 16
64 18
64 51
64 24
64 58
65 0
65 40
65 61
65 37
65 15
66 32
66 1
66 9
66 13
66 16
66 54
66 58
66 60
66 62
67 66
67 35
67 36
67 10
67 34
67 58
67 59
67 26
68 40
68 48
68 46
69 64
69 66
69 4
69 40
69 9
69 20
69 56
69 36
69 28
69 52
70 32
70 65
70 4
70 45
70 18
70 20
70 28
70 30
71 16
71 41
71 13
71 48
71 20
71 27
72 33
72 40
72 12
72 42
72 44
72 47
72 55
73 33
73 5
73 7
73 8
73 9
73 42
73 12
73 47
73 19
73 52
73 22
73 69
74 73
74 50
74 27
74 37
75 64
75 1
75 2
75 33
75 8
75 10
75 71
75 44
75 46
75 29
75 72
75 53
75 27
75 60
75 61
76 2
76 68
76 13
76 14
76 17
76 61
77 0
77 68
77 70
77 39
77 42
77 46
77 17
77 51
77 58
77 31
78 64
78 1
78 66
78 35
78 39
78 75
78 45
78 14
78 15
78 48
78 17
78 67
78 21
78 28
79 64
79 2
79 37
79 7
79 40
79 42
79 16
79 19
79 52
79 58
80 40
80 73
80 79
80 11
80 44
80 45
80 78
80 77
80 48
80 18
80 54
80 57
80 15
80 62
80 63
81 2
81 68
81 69
81 72
81 42
81 12
81 48
81 57
82 65
82 70
82 9
82 75
82 51
82 26
82 58
82 30
83 66
83 38
83 77
83 43
83 50
83 14
83 45
83 82
83 19
83 20
83 54
83 23
83 18
83 55
83 28
84 64
84 5
84 71
84 73
84 39
84 15
84 18
84 51
84 28
84 62
85 27
85 72
85 41
85 43
85 17
85 21
85 55
85 59
85 30
86 38
86 39
86 10
86 13
86 15
86 17
86 42
86 62
87 32
87 2
87 4
87 39
87 74
87 17
87 51
87 21
87 24
87 68
88 19
88 1
88 3
88 4
88 54
88 77
88 18
88 83
88 21
88 86
88 26
88 60
88 85
89 64
89 1
89 3
89 69
89 38
89 65
89 40
89 9
89 41
89 76
89 45
89 46
89 15
89 84
89 21
89 22
89 55
89 47
89 30
89 63
90 64
90 66
90 27
91 64
91 68
91 67
91 4
91 7
91 41
91 39
91 75
91 52
91 23
91 88
91 89
91 87
91 29
92 51
92 67
92 4
92 5
92 39
92 77
92 14
92 83
92 78
92 87
92 24
92 57
92 61
92 30
93 32
93 66
93 3
93 72
93 41
93 23
93 16
93 17
93 51
93 85
93 27
93 87
93 56
93 91
93 63
94 34
94 70
94 71
94 28
94 7
94 80
94 81
94 82
94 51
94 84
94 22
94 25
94 90
94 31
94 92
94 63
95 69
95 6
95 39
95 8
95 42
95 12
95 77
95 46
95 19
95 89
95 58
95 27
95 60
95 30
96 67
96 76
96 10
96 75
96 12
96 78
96 79
96 18
96 84
96 21
96 86
96 88
96 25
96 60
97 32
97 70
97 7
97 40
97 74
97 39
97 47
97 83
97 92
97 62
98 64
98 96
98 36
98 6
98 9
98 49
98 85
98 22
98 89
98 91
98 28
98 62
99 32
99 72
99 76
99 86
99 89
99 25
99 28
100 36
100 40
100 75
100 76
100 77
100 18
100 21
100 89
100 90
100 91'''.split('\n')

In [4]:
edges = [i.split() for i in edges]
edges = pd.DataFrame(edges)
edges = edges.astype(float)
edges = edges.values.tolist()

In [5]:
from collections import defaultdict

graph = defaultdict(list)

In [6]:
for i in edges:
    graph[str(int(i[0]))].append(str(int(i[1])))

In [7]:
for k, v in graph.items():
    print(k, power_generation[k], power_usage[k])
    break

5 0.0 1.002


In [8]:
traversed = []

for k, v in graph.items():
    for node in v:
        print('depth 1')
        if node in graph.keys() and node not in traversed:
            traversed.append(node)
            for node2 in graph.get(node):
                print('depth 2')
                
                
#                 if node2 in graph.keys()and node2 not in traversed:
#                     traversed.append(node2)
#                     for node3 in graph.get(node2):
#                         print('depth 3')
#                         if node3 in graph.keys() and node3 not in traversed:
#                             for node4 in graph.get(node3):
#                                 print('depth 4')
#                                 if node4 in graph.keys():
#                                     for node5 in graph.get(node4):
#                                         print('depth 5')

depth 1
depth 1
depth 1
depth 1
depth 1
depth 2
depth 2
depth 1
depth 1
depth 1
depth 2
depth 1
depth 1
depth 1
depth 2
depth 2
depth 1
depth 2
depth 1
depth 1
depth 2
depth 2
depth 1
depth 2
depth 1
depth 2
depth 1
depth 1
depth 1
depth 2
depth 2
depth 2
depth 1
depth 1
depth 1
depth 1
depth 2
depth 1
depth 2
depth 2
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 2
depth 1
depth 2
depth 2
depth 2
depth 2
depth 1
depth 2
depth 2
depth 1
depth 1
depth 1
depth 2
depth 2
depth 2
depth 1
depth 1
depth 1
depth 1
depth 2
depth 2
depth 2
depth 2
depth 2
depth 1
depth 1
depth 1
depth 2
depth 2
depth 1
depth 1
depth 2
depth 2
depth 2
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 2
depth 1
depth 2
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 1
depth 2
depth 1
depth 2
depth 2
depth 2
depth 2
depth 2
depth 2
depth 2
depth 1
depth 1
depth 2
depth 2


In [13]:
# Calculating power generation

def power_gen_downstream(master_node='100'):
    if isinstance(master_node, int):
        master_node = str(master_node)
        
    traversed = []
    power = 0

    power += float(power_generation[master_node])

    for node in graph[master_node]:
        if node in graph.keys() and node not in traversed:
            traversed.append(node)
            power += float(power_generation[node])
            for node2 in graph.get(node):
                if node2 in graph.keys() and node2 not in traversed:
                    traversed.append(node2)
                    power += float(power_generation[node2])
    
    return power

In [14]:
# Calculate how much power this node and its children consume

def power_usage_downstream(master_node='100'):
    if isinstance(master_node, int):
        master_node = str(master_node)
        
    traversed = []
    power = 0

    power += float(power_usage[master_node])

    for node in graph[master_node]:
        if node in graph.keys() and node not in traversed:
            traversed.append(node)
            power += float(power_usage[node])
            for node2 in graph.get(node):
                if node2 in graph.keys() and node2 not in traversed:
                    traversed.append(node2)
                    power += float(power_usage[node2])
    
    return power

In [17]:
# Returns generation minus consumption

def net_power(node):
    if isinstance(node, int):
        node = str(node)
    generation = power_gen_downstream(node)
    usage = power_usage_downstream(node)
    return generation - usage

In [18]:
net_power('100')

18.066000000000003

In [152]:
for node in graph['100']:
    print(node, power_generation[node])

36 1.452
40 0.0
75 0.0
76 0.0
77 0.0
18 0.0
21 0.0
89 0.0
90 17.707
91 0.0


In [153]:
power

132.36599999999999